### [ Lamp Recol ]

[1] 모듈 로딩 <hr>

In [ ]:
from urllib.request import urlopen
from urllib.request import Request  

from bs4 import BeautifulSoup

import pandas as pd

import selenium 
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait # 특정요소 로딩될떄까지 기다리는거

from selenium.webdriver.support import expected_conditions as EC #요소 클릭, 존재여부
from selenium.common.
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time
import re # 정규표현식

[2] 웹에서 데이터 다운로드

In [42]:
## =========================================================
## Selenium 4.6 이상 드라이버 자동 설치/관리 (Selenium Manager)
## =========================================================
## - 크롬 브라우저 실행
driver = webdriver.Chrome()    

## - 페이지 열기   
driver.get("https://www.car.go.kr/ri/stat/list.do")                 

In [43]:
## 검색창
start_data = driver.find_element(By.NAME, "searchFromDate")
end_date = driver.find_element(By.NAME, "searchToDate")

## 검색어 입력
start_data.send_keys("2014-01-01")
end_date.send_keys("2014-12-31")

## 엔터
end_date.send_keys(Keys.RETURN)

In [44]:
page = 1 

def go_page(p):
    try:
        elem = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.LINK_TEXT, str(p)))
        )
        elem.click()
        time.sleep(1)

    except:
        try:
            next_btn = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "a[title='다음페이지']"))
            )
            next_btn.click()
            
            # 페이지 그룹 변경 후,다시 로딩될 때까지
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.paging"))
            )

            time.sleep(7)
            
            # 재귀함수?
            return go_page(p)
            
        except:
            print("다음페이지 ERROR")


In [45]:
def get_post_count(): 
    return len(driver.find_elements(By.CSS_SELECTOR,
                                    "ul.board-hrznt-list a[href='#']")) 

In [46]:
# 마지막 페이지

def get_last_page():
    btn = driver.find_element(By.CSS_SELECTOR, "a[title='끝페이지']")
    onclick = btn.get_attribute("onclick")

    # 정규식
    m = re.search(r"\((\d+)\)", onclick)
    if m:
        return int(m.group(1))

In [ ]:
date = []
kind = []
content = [] 
solution = [] 

lamp = [] 

last_page = get_last_page()          

# 2) 페이지 1 ~ 마지막페이지까지 전체 반복
        
for page in range(1, last_page+1): 
    print(f"\n=== {page}페이지 시작 ===") 
                
    # 1페이지는 이미 로딩됨 / 2페이지 이상은 클릭해서 이동 
    if page > 1: 
        go_page(page)
        
    post_count = get_post_count()    

    # 해당페이지 나타날 때까지 기다림
    WebDriverWait(driver, 5).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul.board-hrznt-list a[href='#']"))
    )
    # 해당페이지 클릭상태 될 때까지 기다림
    WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "ul.board-hrznt-list a[href='#']"))
    )
    time.sleep(0.5) 
                

    # 3) 게시글 크롤링  
    for i in range(post_count): 
        posts = driver.find_elements(By.CSS_SELECTOR, "ul.board-hrznt-list a[href='#']") 
        posts[i].click() 
        time.sleep(1.5) 
                    
        html_str = driver.page_source 
                    
        BS = BeautifulSoup(html_str, 'html.parser') 
        
        date = BS.select_one("tbody.uk-text-left > tr:nth-child(2) > td:nth-child(4)").get_text()
        kind = BS.select_one("tbody.uk-text-left > tr:nth-child(3) > td:nth-child(4)").get_text()
        content = BS.select_one("tbody.uk-text-left > tr:nth-child(4) > td:nth-child(2)").get_text()
        solution = BS.select_one("tbody.uk-text-left > tr:nth-child(5) > td:nth-child(2)").get_text()

                    
        #DF에 넣기 쉽게 리스트로
        lamp.append([date, kind, content, solution]) 
                    
        driver.back() 
        time.sleep(1.5)
                


=== 1페이지 시작 ===

=== 2페이지 시작 ===

=== 3페이지 시작 ===

=== 4페이지 시작 ===

=== 5페이지 시작 ===


InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=142.0.7444.176); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff61406a235
	0x7ff613dc2630
	0x7ff613b516dd
	0x7ff613b3d205
	0x7ff613b62a8a
	0x7ff613bda476
	0x7ff613bfb3d2
	0x7ff613b9b0ed
	0x7ff613b9bf63
	0x7ff614095d60
	0x7ff61408fe8a
	0x7ff6140b1005
	0x7ff613ddd71e
	0x7ff613de4e1f
	0x7ff613dcb7c4
	0x7ff613dcb97f
	0x7ff613db18e8
	0x7ff994bee8d7
	0x7ff995bac53c


In [ ]:
## 데이터 csv파일로 저장
DF  = pd.DataFrame(lamp, columns=["date", "kind", "content", "solution"])
DF.to_csv("All_recol.csv", index=False, encoding="utf-8-sig")